In [ ]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import skimage.transform
from keras.models import model_from_json, load_model

In [ ]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    
    mod = ds.Modality
#     pos = ds.PatientSetupSequence[0].PatientPosition
    pos = ds.PatientPosition
    bod = ds.BodyPartExamined
    
    if mod != 'DX': 
        print('Incorrect modality detected')
        return
        
#     if ~pos.isin(['PA','AP']):
    if pos not in ['PA','AP']:
        print('Incorrect patient position detected')
        return 
    
    if bod != 'CHEST':
        print('Incorrect body part examined')
        return
            
#     img = dataset.pixel_array
    img = ds.pixel_array
    return img
  
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    
    proc_img = (img-img_mean)/img_std
    proc_img = skimage.transform.resize(proc_img,img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    
    with open(model_path, 'r') as f:
        model = model_from_json(f.read())
#     model = keras.models.load_model(model_path)
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    
    pred = model.predict(img)
    prediction = 'No Pnuemonia'
    
    if pred > thresh:
        prediction = 'Pneumonia'
    
    return prediction

In [ ]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json' #path to student's saved model
weight_path = 'xray_class_vgg16_binary.best.hdf5'#path to student's saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if the student did not use vgg16
img_mean = 134 # student loads the mean image value they used during training preprocessing
img_std = 65# student loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = 0.42356247 #student loads the threshold they chose for model classification 

for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
#     if img.size == 0:
#         continue
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

### Correct solution is:
No Pneumonia

No Pneuomonia

Pneumonia

Incorrect body part examined

Incorrect modality detected

Incorrect patient position detected